In [6]:
from src.Import_Patent_Data import import_data
#%load Import_Patent_Data.py 
from src.Clean_Abstracts import *
from src.Tfidf_on_Pandas import *
from src.NMF_Analysis import *
import matplotlib.pyplot as plt


import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()


from sklearn.decomposition import LatentDirichletAllocation



In [2]:
df = pd.read_csv("data/temp/pd_2018_and_later.csv", sep="|")

In [3]:

df['abstract_lemmed'] = df['abstract_lemmed'].str[1:-1]
df['abstract_lemmed'] = df['abstract_lemmed'].str.replace("'" ,"")
df['abstract_lemmed'] = df['abstract_lemmed'].str.split(", ")
df['abstract_lemmed_stemmed'] = df['abstract_lemmed_stemmed'].str[1:-1]
df['abstract_lemmed_stemmed'] = df['abstract_lemmed_stemmed'].str.replace("'" ,"")
df['abstract_lemmed_stemmed'] = df['abstract_lemmed_stemmed'].str.split(", ")

In [4]:
#tfidf_matrix = tfidf_vector_matrix(df,2,2,'abstract_lemmed_stemmed')

In [10]:
corpus = [" ".join(row) for row in df['abstract_lemmed_stemmed']]
tfidf = TfidfVectorizer(ngram_range=(2,2),min_df=.01,max_df=.6)
tfidf_matrix = tfidf.fit_transform(np.array(corpus))


In [9]:

# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=7, random_state=0)
lda_tfidf.fit(tfidf_matrix)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=7, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [14]:
p = pyLDAvis.sklearn.prepare(lda_tfidf, np.matrix(tfidf_matrix), tfidf)

/opt/conda/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [15]:

pyLDAvis.save_html(p, 'lda.html')

In [16]:
import gensim
# the model has 500 dimensions, the minimum document-term frequency is 20
w2v_model = gensim.models.Word2Vec(df['abstract_lemmed_stemmed'], size=500, min_count=2)


/opt/conda/lib/python3.7/site-packages/gensim/corpora/dictionary.py:11: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict
/opt/conda/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [17]:
print( "Model has %d terms" % len(w2v_model.wv.vocab) )

Model has 41918 terms


In [ ]:
def calculate_coherence( w2v_model, term_rankings ):
    overall_coherence = 0.0
    for topic_index in range(len(term_rankings)):
        # check each pair of terms
        pair_scores = []
        for pair in combinations( term_rankings[topic_index], 2 ):
            pair_scores.append( w2v_model.similarity(pair[0], pair[1]) )
        # get the mean for all pairs in this topic
        topic_score = sum(pair_scores) / len(pair_scores)
        overall_coherence += topic_score
    # get the mean score across all topics
    return overall_coherence / len(term_rankings)